In [3]:
import json
import os
import django
import sys
sys.path.append('..')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
django.setup()
from Manage.models import College,Term,Branch,Stream,Semester,Division,Batch,Subject,TimeTable,GPSCoordinates,Classroom,Schedule,Lecture,Link
from Session.models import Attendance,Session
from StakeHolders.models import NotificationSubscriptions,SuperAdmin,Admin,Teacher,Student,Profile
from django.contrib.auth import get_user_model

In [4]:
folder_path = '../student-sheets'
files_with_paths = [os.path.abspath(os.path.join(folder_path, f)) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

In [7]:
def create_student_obj(branch,stream_code,sr_no,enrollment,name):    
    stream_obj = Branch.objects.filter(slug=branch).first().stream_set.filter(stream_code=stream_code).first()
    if stream_obj:
        name = name.rstrip()
        student_obj,student_created = Student.objects.get_or_create(enrollment=enrollment,sr_no=sr_no)
        if student_obj:    
            profile_obj,created = Profile.objects.get_or_create_by_name(name=name,role='student')
            student_obj.profile = profile_obj
            student_obj.save()
            stream_obj.students.add(student_obj)
            return student_obj
        return None

In [12]:
file = open('../student-sheets/3rd_AIML_Students.json')
data = json.load(file)
for division,division_data in data.items():
        for batch,batch_data in division_data.items():
            for student in batch_data:
                student_obj = create_student_obj(branch='332561_1731147390',stream_code='52',sr_no=student['sr_no'],enrollment=student['enrollment'],name=student['name'])
                if student_obj:
                    print(f"Student created {student_obj}")
                else:
                    print(f"Student not created {student}")

Student created AGRAHARI PANKAJ HUKUMACHANDRA
Student created Alani Manoj Bharatbhai
Student created Ankush Chunilal Ratnani
Student created Atharva Jitendra Dholakia
Student created Balar Maulik Ghanshyambhai
Student created BAMBHANIYA BRIJESHKUMAR SANABHAI
Student created Bhammar Krunal Jethurbhai
Student created Bhatt Priyan Naman
Student created BUHA VASU MUKESHBHAI
Student created CHAVDA SHAKTISINH DHIRUBHA
Student created DASADIYA MESHWA HARIKRUSHNABHAI
Student created DASALANI JAY NATHALAL
Student created Desai Anand Kalyanbhai
Student created DESAI AYUSHI DILIPBHAI
Student created Dharati Chovatiya
Student created DHARAV ANTANI
Student created Dhaval
Student created DHOLAKIYA MAYUR ASHOKBHAI
Student created divyesh padsala
Student created GAJERA KULDIP VIPULBHAI
Student created GANVIT BHUMIBEN MAHESHBHAI
Student created GURJAR MAHENDRA NAVALBHAI
Student created ITALIYA NEEL PANKAJBHAI
Student created Jogadiya anand
Student created Kachhadiya Vedant Alpeshbhai
Student created Ka